In [ ]:
import pandas as pd

In [ ]:
sales_data = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/f_sales_v_fashion_with_price_126_stores.csv')
sales_data['date'] = pd.to_datetime(sales_data['date'])
sales_data['item'] = sales_data['sku'].astype(str)
sales_data['store'] = sales_data['store'].astype(str)
sales_data = sales_data.rename(columns={'total_sales':'sales'})
sales_data

In [ ]:
mbew_fashion = pd.read_csv('mbew_fashion_126_stores.csv')
mbew_fashion['valid_to_date'] = mbew_fashion['valid_to_date'].replace('9999-12-31', sales_data['date'].max())
mbew_fashion['valid_to_date'] = pd.to_datetime(mbew_fashion['valid_to_date'])
mbew_fashion['valid_from_date'] = pd.to_datetime(mbew_fashion['valid_from_date'])
mbew_fashion['item'] = mbew_fashion['sku'].astype(str).str[:12]
mbew_fashion

In [ ]:
sales_data.columns

In [ ]:
AAA = [3, 18, 22, 28, 29, 44, 51, 57, 63, 73, 74, 76, 79, 88, 89, 91, 96, 100, 117, 119, 123, 130, 133, 135, 136, 141, 143, 144, 149, 150, 152, 162, 164, 166, 167, 168, 171, 172, 175, 179, 181, 184, 185, 186, 188, 189, 202, 214, 216, 217, 226, 3005, 3202, 3205, 3208, 4104, 4123, 4129, 4134, 4803, 4805, 4904, 4906, 4, 5, 7, 8, 10, 11, 15, 21, 26, 27, 35, 36, 37, 42, 43, 45, 46, 47, 50, 52, 55, 56, 61, 64, 67, 68, 69, 81, 82, 84, 85, 90, 95, 99, 104, 105, 106, 109, 114, 121, 122, 147, 156, 159, 160, 163, 170, 173, 174, 180, 182, 183, 201, 203, 213, 215, 218, 219, 220, 221, 225, 3245, 4133]
#AAA to str
AAA  = [str(x) for x in AAA]

In [ ]:
print(len(AAA))

In [ ]:
sales_data = sales_data[sales_data['fashiongrade'] == 'M']

In [ ]:
sales_data['item'] = sales_data['sku'].astype(str).str[:12]

In [ ]:
sales_data_g_item = sales_data.groupby(['date','item'])['sales'].sum().reset_index()

In [ ]:
sales_data_g_item['date'] = pd.to_datetime(sales_data_g_item['date'])

In [ ]:
# sales of sum for each item in 2023 that are large then 0 and fashiongrade is 'M'
items_sales_in_2023 = sales_data_g_item[sales_data_g_item['date'].dt.year == 2023].groupby('item')['sales'].sum().sort_values(ascending=False)[sales_data_g_item[sales_data_g_item['date'].dt.year == 2023].groupby('item')['sales'].sum().sort_values(ascending=False) > 100].index


In [ ]:
import matplotlib.pyplot as plt
for item in items_sales_in_2023[50:100]:
    item_data = sales_data_g_item[sales_data_g_item['item'] == item].groupby('date')['sales'].sum()
    fashiongrade = sales_data[sales_data['item'] == item]['fashiongrade'].unique()[0]
    # reindex to fill in missing dates
    item_data = item_data.reindex(pd.date_range(start=sales_data_g_item['date'].min(),
                                                # today
                                                end=sales_data_g_item['date'].max(),

                                                freq='D')).fillna(0)
    item_data_w = item_data.resample('W').sum()
    # fig size
    plt.figure(figsize=(20,4))
    plt.title('item: ' + item + ' fashiongrade: ' + str(fashiongrade))
    item_data_w.plot()
    plt.show()

In [ ]:
for store in sales_data['store'].unique()[:1]:
    store_sales = sales_data[sales_data['store'] == store]
    for item in store_sales['item'].unique()[:1]:
        item_store_sales = store_sales[store_sales['item'] == item].groupby('date').agg({'average_price':'mean',
                                                                                'average_dicounted_price':'mean',
                                                                                'sales':'sum'})
        item_store_sales = item_store_sales.reindex(pd.date_range(start=item_store_sales.index.min(),
                                                    end=item_store_sales.index.max(),
                                                    freq='D'))
        item_store_sales['average_price'] = item_store_sales['average_price'].fillna(method='ffill')
        item_store_sales['average_dicounted_price'] = item_store_sales['average_dicounted_price'].fillna(method='ffill')
        item_store_sales['sales'] = item_store_sales['sales'].fillna(0)

        item_store_sales.plot(figsize=(20,4))
        plt.title('item: ' + item + ' store: ' + str(store))
        plt.show()

In [ ]:
for item in ['100652103000']:
    item_data = sales_data[sales_data['item'] == item].groupby('date').agg({'average_price':'mean',
                                                                            'average_dicounted_price':'mean',
                                                                            'sales':'sum'})
    item_data = item_data.reindex(pd.date_range(start=item_data.index.min(),
                                                end=item_data.index.max(),
                                                freq='D'))
    item_data['average_price'] = item_data['average_price'].fillna(method='ffill')
    item_data['average_dicounted_price'] = item_data['average_dicounted_price'].fillna(method='ffill')
    item_data['sales'] = item_data['sales'].fillna(0)
    item_data.plot(figsize=(20,4))
    plt.title('item: ' + item)
    plt.show()


In [ ]:
sales_data[sales_data['item'] == '100652103000']['sku'].unique().tolist()

In [ ]:
store_sales['store'].unique()

In [ ]:


for store in sales_data['store'].unique()[:10]:
    store_sales = sales_data[sales_data['store'] == store]
    store_stock = mbew_fashion[mbew_fashion['store'].astype(str) == store]
    for sku in [100652103000001, 100652103000002, 100652103000003, 100652103000004]:
        stock_sku_store_data = pd.DataFrame()
        sku_store_stock = store_stock[store_stock['sku'].astype(str) == str(sku)]
        for row in sku_store_stock.iterrows():
            stock_sku_store_data = pd.concat([stock_sku_store_data,
                                            pd.DataFrame({'date':pd.date_range(start=row[1]['valid_from_date'],
                                                                                end=row[1]['valid_to_date'],
                                                                                freq='D'),
                                                        'stock':row[1]['stock']})])
        if stock_sku_store_data.empty:
            continue
        stock_sku_store_data = stock_sku_store_data.set_index('date')
        sku_store_sales = store_sales[store_sales['sku'] == sku].groupby('date').agg({'average_price':'mean',
                                                                                'average_dicounted_price':'mean',
                                                                                'sales':'sum'})
        if sku_store_sales.empty:
            continue
        sku_store_sales = sku_store_sales.reindex(pd.date_range(start=sku_store_sales.index.min(),
                                                    end=sku_store_sales.index.max(),
                                                    freq='D'))
        sku_store_sales['average_price'] = sku_store_sales['average_price'].fillna(method='ffill')
        sku_store_sales['average_dicounted_price'] = sku_store_sales['average_dicounted_price'].fillna(method='ffill')
        sku_store_sales['sales'] = sku_store_sales['sales'].fillna(0)
        sku_store_sales = sku_store_sales.merge(stock_sku_store_data, left_index=True, right_index=True)
        df = sku_store_sales
        # Plotting
        fig, ax1 = plt.subplots(figsize=(20, 4))

        # Left y-axis for prices
        ax1.plot(df.index, df['average_price'], label='Average Price', color='b')
        ax1.plot(df.index, df['average_dicounted_price'], label='Average Discounted Price', color='g')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Price ($)', color='g')
        ax1.tick_params(axis='y', labelcolor='g')
        ax1.legend(loc='upper left')

        # Right y-axis for quantities
        ax2 = ax1.twinx()
        ax2.plot(df.index, df['sales'], label='Sales', color='r')
        ax2.plot(df.index, df['stock'], label='Stock', color='purple')
        ax2.set_ylabel('Quantity', color='r')
        ax2.tick_params(axis='y', labelcolor='r')
        ax2.legend(loc='upper right')

        # Title
        plt.title('SKU Sales and Stock Analysis, Store: ' + str(store) + ' SKU: ' + str(sku))

        plt.show()

In [ ]:
sku_store_sales

In [36]:
sales_data.groupby('date')['sales'].sum().plot()


,store,sku,valid_from_date,valid_to_date,stock,fashiongrade
0,VZ01,100552720000004,2019-10-01,2099-12-31,0.0,M
1,VZ01,100550173000003,2018-03-27,2018-03-27,62.0,M
2,VZ01,100552654000001,2018-03-27,2018-03-27,160.0,M
3,VZ01,100552654000002,2018-03-27,2018-03-27,165.0,M
4,VZ01,100510035000011,2018-03-27,2018-03-27,55.0,M
...,...,...,...,...,...,...
1370476,VZ01,100628362000011,2023-06-17,2099-12-31,64.0,M
1370477,VZ01,100628362000008,2023-06-17,2099-12-31,60.0,M
1370478,VZ01,100620362000004,2023-06-17,2099-12-31,154.0,M
1370479,VZ01,100610362000003,2023-06-17,2099-12-31,100.0,M
